In [1]:
import os

In [4]:
%pwd

'g:\\End-to-End-ML-Project\\End-to-end-Ml-project'

In [3]:
os.chdir('../')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen = True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [6]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    
    def __init__(self, config_pathfile = CONFIG_PATH_FILE, schema_pathfile = SCHEMA_PATH_FILE, 
                 params_pathfile = PARMAS_PATH_FILE):      
            
        self.config = read_yaml(config_pathfile) 
        self.params = read_yaml(params_pathfile)
        self.schema = read_yaml(schema_pathfile)

        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir= config.unzip_data_dir,
            STATUS_FILE= config.STATUS_FILE,
            all_schema=schema
        )
        return data_validation_config

In [9]:
from mlproject import logger
import pandas as pd

In [48]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_columns = data.columns

            all_schema = self.config.all_schema.keys()
            for cols in all_columns:
                if cols not in all_schema:
                    validation_status= False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"Validation Status for column: {cols} is {validation_status}\n")
            
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"Validation Status for column: {cols} is {validation_status}\n")
                
            return validation_status
        
        except Exception as e:
            raise e

    def validate_all_dtypes(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_columns = data.columns
            data_type = data.dtypes

            all_schema = self.config.all_schema
            for cols in all_columns:
                if data_type[cols].name == all_schema[cols]:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"Validation Status for column: {cols} with datatype: {data_type[cols]} is {validation_status}\n")
                    
                else:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"Validation Status for column: {cols} with datatype: {data_type[cols]} is {validation_status}\n")
                        
            return validation_status
        
        except Exception as e:
            raise e

In [47]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_all_dtypes()
except Exception as e:
    raise e

[2024-01-08 00:22:27,293: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-08 00:22:27,300: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-08 00:22:27,333: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-08 00:22:27,342: INFO: common: created directory at: artifacts]
[2024-01-08 00:22:27,345: INFO: common: created directory at: artifacts/data_validation]


False